In [2]:
from pyspark import SparkConf, SparkContext

# Get existing SparkContext if available or create a new one
try:
    sc = SparkContext.getOrCreate()
    print("Using existing SparkContext")
except:
    conf = SparkConf().setMaster("local").setAppName("MinTemperatures")
    sc = SparkContext(conf=conf)
    print("Created new SparkContext")


def parseLine(line):
    fields = line.split(",")
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)


lines = sc.textFile("./data/1800.csv")
parsedLines = lines.map(parseLine)
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: min(x, y))
results = minTemps.collect()
for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

sc.stop()

Using existing SparkContext


ITE00100554	5.36F
EZE00100082	7.70F
